## Importing packages

In [115]:
'''# Record of additions to Jupyter docker's Python packages:
pip install --upgrade pip
pip install lxml
pip install pyxDamerauLevenshtein
pip install google
pip install https://github.com/slimkrazy/python-google-places/zipball/master'''

"# Record of additions to Jupyter docker's Python packages:\npip install --upgrade pip\npip install lxml\npip install pyxDamerauLevenshtein\npip install google\npip install https://github.com/slimkrazy/python-google-places/zipball/master"

In [116]:
from bs4 import BeautifulSoup
import pandas, csv, numpy as np
import urllib, requests
import os, os.path, re
from glob import glob

# for comparing strings in order to match elements between lists:
from pyxdameraulevenshtein import normalized_damerau_levenshtein_distance as dldist
from pyxdameraulevenshtein import normalized_damerau_levenshtein_distance_ndarray as dldistarray

#from lxml import html, etree
# from requests.auth import HTTPBasicAuth, HTTPDigestAuth
# import nbimporter #need to pip install this
# import EncapsulatedSchoolObject as school

In [117]:
print(dldist('smtih', 'smith'))  # expected result: 0.2

array = np.array(['test1', 'test12', 'test123'])
print('\n', dldistarray('test', array))  # expected result: [0.2, 0.33333334, 0.42857143]
print('\n', min(dldistarray('test', array)))

0.20000000298023224

 [ 0.2         0.33333334  0.42857143]

 0.2


## Reading in data

In [118]:
sample = [] # make empty list to store the dictionaries in
with open('../web_scraping/charter_URLs_Apr17.csv', 'r', encoding = 'Latin-1')\
as csvfile: # open file                      
    reader = csv.DictReader(csvfile) # create a reader
    for row in reader: # loop through rows
        sample.append(row) # append each row to the list

In [119]:
# Take a look at the first entry's contents and the variables list in our sample (a list of dictionaries)
print(sample[1]['SEARCH'], "\n", sample[1]["OLD_URL"], "\n")
print(sample[1].keys())

POLK STATE COLLEGE COLLEGIATE HIGH SCHOOL 3425 WINTER LK RD LAC1200, WINTER HAVEN, FL 33881 
 https://www.polk.edu/charter-high-schools/ 

odict_keys(['SEARCH', 'MANUAL_URL', 'OLD_URL', 'ADDRESS', 'STABR', 'NCESSCH', 'SCH_NAME'])


In [120]:
'''# read in data
df = pandas.read_excel('micro-sample_MANUAL_Apr17_rev3.xlsx')

#get the values from the URL column (stored as list)
links = df['URL'].values

# Input test URL
url = links[0]
url
'''

"# read in data\ndf = pandas.read_excel('micro-sample_MANUAL_Apr17_rev3.xlsx')\n\n#get the values from the URL column (stored as list)\nlinks = df['URL'].values\n\n# Input test URL\nurl = links[0]\nurl\n"

In [121]:
'''for i in $(find -maxdepth 1 -type d); do echo -n "$i "; find $i -type f | wc -l; done  | grep ' 1$'  | wc -l
   55  find www.polk.edu/
   56  find | head
   57  find | head -n 100
   58  find
   59  find www.hustonacademy.com/
   60  find www.hustonacademy.com/ -type d
   61  find www.hustonacademy.com/ -maxdepth 1 -type d
   62  find www.hustonacademy.com/ -maxdepth 0 -type d
   63  find www.hustonacademy.com/ -maxdepth 1 -type d
   64  find www.hustonacademy.com/ -maxdepth 2 -type d
   65  find www.hustonacademy.com/ -maxdepth 3 -type d
   66  find -maxdepth 1 -type d
   67  l s-la
   68  ls -la
   69  for i in $(find -maxdepth 1 -type d); do echo -n "$i "; find $i -type f | wc -l; done  | grep ' 1$'  | wc -l
   70  for i in $(find -maxdepth 1 -type d); do echo -n "$i "; find $i -type f | wc -l; done  | grep ' 1$'  
   71  for i in $(find -maxdepth 1 -type d); do echo -n "$i "; find $i -type f | wc -l; done  | grep -v ' 1$'

!find .  -maxdepth 1 -type d | wc -l

!find /tmp  -maxdepth 1 -type d | wc -l'''

'for i in $(find -maxdepth 1 -type d); do echo -n "$i "; find $i -type f | wc -l; done  | grep \' 1$\'  | wc -l\n   55  find www.polk.edu/\n   56  find | head\n   57  find | head -n 100\n   58  find\n   59  find www.hustonacademy.com/\n   60  find www.hustonacademy.com/ -type d\n   61  find www.hustonacademy.com/ -maxdepth 1 -type d\n   62  find www.hustonacademy.com/ -maxdepth 0 -type d\n   63  find www.hustonacademy.com/ -maxdepth 1 -type d\n   64  find www.hustonacademy.com/ -maxdepth 2 -type d\n   65  find www.hustonacademy.com/ -maxdepth 3 -type d\n   66  find -maxdepth 1 -type d\n   67  l s-la\n   68  ls -la\n   69  for i in $(find -maxdepth 1 -type d); do echo -n "$i "; find $i -type f | wc -l; done  | grep \' 1$\'  | wc -l\n   70  for i in $(find -maxdepth 1 -type d); do echo -n "$i "; find $i -type f | wc -l; done  | grep \' 1$\'  \n   71  for i in $(find -maxdepth 1 -type d); do echo -n "$i "; find $i -type f | wc -l; done  | grep -v \' 1$\'\n\n!find .  -maxdepth 1 -type d | 

## Configuring environment for parsing text

In [122]:
data_loc = "/Users/bradafzali/research/text_analysis/wget_sept8/" # Point to where HTML has been captured

In [123]:
# For different ways of parsing the text, identify elements we know we DON'T want (to rip out):
junkName = ['style', 'script', '[document]', 'head', 'title', 'meta', 'a', 'label', 'footer']

# And also elements we know we DO want (to keep):
correctName = ['p', 'li', 'table', 'ul', 'ol', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'h7', 'h8', 'h9', 'h10']

In [124]:
os.getcwd()

'/Users/bradafzali/research/text_analysis'

In [125]:
# testurl = "https://acaciamiddle.org/about/our-mission-and-philosophy"

try:
    testfile = data_loc + "012 Acacia Middle Charter/acaciamiddle.org/about/our-mission-and-philosophy"
    testfolder = data_loc + "012 Acacia Middle Charter/acaciamiddle.org/"
#     testfile = "/home/jovyan/work/mirror/acaciamiddle.org/about/our-mission-and-philosophy.html"
#     testfolder = "/home/jovyan/work/mirror/acaciamiddle.org/"
    soup = BeautifulSoup(open(testfile), "lxml")
except:
    html_page = urllib.request.urlopen(testurl)
    soup = BeautifulSoup(html_page, "lxml")

ptags = soup.find_all("p")
print(ptags)

[<p>The mission of Acacia Middle Charter School is to provide a nurturing and challenging learning community to the greater Stockton area. Students will experience learning in ways that will allow them to both understand and improve their world. Our exceptional teachers inspire students to seek deep, meaningful understanding; to develop strong academic skills; and to become creative and independent thinkers. A rapidly changing global environment requires that Acacia students acquire knowledge and develop 21<sup>st</sup> century skills, while learning to define and solve problems creatively. Acacia Middle cultivates compassion, fairness, respect, and individuality within a community that celebrates diversity, authenticity, and service.</p>, <p>Acacia Middle embodies the following key assumptions in philosophy and daily practice:</p>, <p> </p>, <p> </p>, <p> </p>, <p>©Tri Valley Learning Corporation - All Rights Reserved </p>, <p>TVLC and the charter schools it operates, Livermore Valley

In [126]:
# This gives only the VISIBLE text on the web page:

from bs4.element import Comment
data = soup.findAll(text=True) 

def visible(element):
    if element.parent.name in ['style', 'script', '[document]', 'head', 'title', 'meta']:
        return False
    elif isinstance(element, Comment):
        return False
    return True

# Alternative way of ripping out comments (line 3 of "visible" function above):
# elif re.match('<!--.*-->', str(element.encode('utf-8'))):

visible_texts = list(filter(visible, data))
visible_texts = [el for el in visible_texts if el[0] != '\n' and el.strip() != ""]
v_t = ""
for el in visible_texts:
    v_t += el + " "
# v_t.replace(u'\xa0', u' ')
v_t

"Skip to main content English Español Staff Facility Support Tech Support Search form Main menu Home About Welcome Principal's Message Mission & Philosophy Our Team FAQs Directions School Wellness Policy Title III LEP Plan LCAP Academics Moby Max Math 180 Read 180 Typing Club CAASPP - California Assessment Student Life Student Life Community Service For Parents SELPA Update SARC Attendance Policy Uniform Policy PowerSchool School Site Council Volunteer Opportunities Family Handbook Common Core Schools LVCS (K-8) LVCP AECS Support Us Support Overview PTC Employer Matching Contact Us Our Mission and Philosophy Language  Our Mission: The mission of Acacia Middle Charter School is to provide a nurturing and challenging learning community to the greater Stockton area. Students will experience learning in ways that will allow them to both understand and improve their world.\xa0Our exceptional teachers inspire students to seek deep, meaningful understanding; to develop strong academic skills;

In [127]:
def format_string(newstring):
    
    '''This removes extraneous characters "b" and "'" at beginning and end of strings.
    Useful to parsing text scraped from HTML files.'''
    
    if   (newstring[:2] == "b'" and
          newstring[-1] == "'"):      
          newstring = newstring[2:-1] #re.sub("b'.*?", "", newstring[:-1])
            
    elif (newstring[0] == "'" and
          newstring[-1] == "'"):
          newstring = newstring[1:-1]
            
    elif (newstring[0] == "'"):
          newstring = newstring[1:]
            
    elif (newstring[-1] == "'"):
          newstring = newstring[:-1]
            
    elif (newstring[:2] == "b'"):
          newstring = newstring[2:]
    
    elif   (newstring[:2] == 'b"' and
          newstring[-1] == '"'):      
          newstring = newstring[2:-1] #re.sub("b'.*?", "", newstring[:-1])
            
    elif (newstring[0] == '"' and
          newstring[-1] == '"'):
          newstring = newstring[1:-1]
            
    elif (newstring[0] == '"'):
          newstring = newstring[1:]
            
    elif (newstring[-1] == '"'):
          newstring = newstring[:-1]
            
    elif (newstring[:2] == 'b"'):
          newstring = newstring[2:]
    
    else:
        pass
    
    return(newstring)

In [128]:
def rip_text(fname, junk_list):

    '''This method of grabbing the visible text from a .html file first converts the HTML to a BeautifulSoup object, 
    then it rips out all script and style elements with tags identified through junkName_list. 
    It finishes by taking out the newlines and outputting a single string of all the visible text from the .html file.'''

    soup = BeautifulSoup(open(fname), "lxml")

    for script in soup(junk_list):   # loop through every junk tag in soup
        script.extract()    # rip out all script and style elements

    ripped_text = soup.get_text() # get text
    # print(ripped_text.encode('utf-8'))   # for checking work

    # break into lines and remove leading and trailing space on each:
    lines = (line.strip() for line in ripped_text.splitlines())
    # break multi-headlines into a line each:
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    ripped_textc = ' '.join(chunk for chunk in chunks if chunk)

    return(ripped_textc.encode('utf-8'))   # return output

In [129]:
# Testing out the parsing and formatting functions:
testtext = format_string(str(rip_text(testfile, junkName)))
print(testtext) # This looks good!

Search form Main menu Our Mission and Philosophy English Our Mission: The mission of Acacia Middle Charter School is to provide a nurturing and challenging learning community to the greater Stockton area. Students will experience learning in ways that will allow them to both understand and improve their world.\xc2\xa0Our exceptional teachers inspire students to seek deep, meaningful understanding; to develop strong academic skills; and to become creative and independent thinkers. A rapidly changing global environment requires that Acacia students acquire knowledge and develop 21st century skills, while learning to define and solve problems creatively. Acacia Middle cultivates compassion, fairness, respect, and individuality within a community that celebrates diversity, authenticity, and service. Our Philosophy: Acacia Middle embodies the following key assumptions in philosophy and daily practice: learning is the young person\xe2\x80\x99s pathway to personal growth school should be both

In [137]:
def keep_text(fname, correctName_list):

    '''This method of grabbing the visible text from a .html file first converts the HTML to a BeautifulSoup object, 
    then it keeps only those elements with tags identified through correctName_list. 
    It finishes by taking out the newlines and outputting a single string of all the visible text from the .html file.'''

    soup = BeautifulSoup(open(fname), "lxml")

    # Iterate through all white listed tags in correctName
    # For each tag that we want to keep, find all text for that tag and append it to good_text
    good_text = ""
    for tag in correctName:
        elements = soup.find_all(tag)
        for el in elements:
            good_text += el.get_text(" ") + " "
    good_text = good_text[:-1]
    
    # Removing annoying bytes just in case
    good_text = good_text.replace(u'\xa0', u'')
    good_text = good_text.replace(u'\u200b', u'')
    good_text = good_text.replace(u'\n', u'')
    
    # break into lines and remove leading and trailing space on each:
    lines = (line.strip() for line in good_text.splitlines())
    # break multi-headlines into a line each:
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    good_textc = ' '.join(chunk for chunk in chunks if chunk)
    
    return good_textc

In [138]:
keep_text(testfile, correctName)

"The mission of Acacia Middle Charter School is to provide a nurturing and challenging learning community to the greater Stockton area. Students will experience learning in ways that will allow them to both understand and improve their world.Our exceptional teachers inspire students to seek deep, meaningful understanding; to develop strong academic skills; and to become creative and independent thinkers. A rapidly changing global environment requires that Acacia students acquire knowledge and develop 21 st century skills, while learning to define and solve problems creatively. Acacia Middle cultivates compassion, fairness, respect, and individuality within a community that celebrates diversity, authenticity, and service. Acacia Middle embodies the following key assumptions in philosophy and daily practice: ©Tri Valley Learning Corporation - All Rights Reserved TVLC and the charter schools it operates, Livermore Valley Charter Preparatory High School, Livermore Valley Charter School, Ac

In [139]:
# keep_text(data_loc + "001 Richland Two Charter High/www.richland2.org/charterhigh/index.html", correctName)

In [140]:
bool(BeautifulSoup(open(testfile), "html.parser").find())==True

True

In [141]:
folder_list = next(os.walk(data_loc))[1]
print(len(folder_list))
folder_list

# list(set(folder_list).intersection(URL_list))

300


['001 Richland Two Charter High',
 '002 Polk State College Collegiate High School',
 '003 River City Scholars Charter Academy',
 '004 Detroit Enterprise Academy',
 '005 Lighthouse Community Sch Inc',
 '006 Westlake Charter Middle',
 '007 Van Gogh Charter',
 '008 Summit Academy Transition High School Dayton',
 '009 Westchester Academy For International Studies',
 '010 City Academy',
 '011 Byrneville Elementary School, Inc.',
 '012 Acacia Middle Charter',
 '013 Whittier Elementary',
 '014 Zoe Learning Acad - Ambassador Campus',
 '015 Young Womens Leadership Chartr Hs',
 '016 Yuba County Career Preparatory Charter',
 '017 Young Scholars Of Western Pennsylvania C',
 '018 West Phila. Achievement Ces',
 '019 John H Wood Jr Cs Granbury',
 '020 Willow Creek Charter School',
 '021 White Pine Academy',
 '022 West Ridge Academy',
 '023 Watts Learning Center Charter Middle',
 '024 Waterford Montessori Academy',
 '025 Valley Life Charter',
 '026 Valley Charter Elementary',
 '027 Virtual Academy Of 

In [142]:
def exact_matching_folder(match_var, list_folders):
    
    reformatted_list_folders = list_folders.copy()
    # first reformat list_folders to allow for easier matching
    for i in range(len(reformatted_list_folders)):
        reformatted_list_folders[i] = reformatted_list_folders[i][4:].lower()
    match_var = match_var.lower()
    
    for i in range(len(reformatted_list_folders)):
        if match_var == reformatted_list_folders[i]:
            return list_folders[i]
    return ""

In [143]:
def matching_folder(data_location, match_var, list_folders):
    
    '''This function finds the folder name in data_loc closest to match_var.
    It matches based on normalized character differences, using the pyxDamerauLevenshtein library.
    This works whether the folders have been named according to SCH_NAME, URL, or something else.'''
    
    # Make array of normalized distances between match_var and each directory name
    distarray = dldistarray(match_var, np.array(list_folders))
    
    # Find index of that array element with the lowest normalized distance to match_var, i.e. index of the best match
    #print("Finding best match out of " + str(len(distarray)) + " options...")
    index_match = list(distarray).index(min(distarray))
    
    # TO DO: If min(distarray)>.5, then search using the given match_var WITHOUT 
    # everything past the last "/" (until str following "http://")
    
    # Use index_match to identify folder in list_folders that best matches match_var
    name_folder = list_folders[index_match]
    
    # Remove result from list_folders in order to return UNIQUE matches--currently low reliability
    # print("Removing " + name_folder + " from list of folders...\n")
    # list_folders.remove(name_folder)
    
    # Return best match, with a printed accuracy check of matching algorithm
    print("Best folder match is " + str(name_folder) + " with normalized distance of " + str(min(distarray)) + "!\n")
    return(name_folder)

In [144]:
# Make list of directories (containing HTML) in data_location
folder_list = next(os.walk(data_loc))[1] 

# Assign a folder to each school in sample using the matchin_folder function

k=0 # initialize iterator

for school in sample: # loop through sample
    print("Finding folder for " + str(k) + " of " + str(len(sample)) + ", " + school["SCH_NAME"] + ", \nlisted " + str(school["MANUAL_URL"]) + "...")
    
    if school["SCH_NAME"] != '':
        
        try:
            school["folder_name"] = exact_matching_folder(school["SCH_NAME"], folder_list)
            print("Found this folder:", school["folder_name"])
            print()
        except Exception as e:
            print("ERROR!")
            school["folder_name"] = ''
    
    else:
        print("Nothing to match on!")
        school["folder_name"] = ''
    
    k+=1

Finding folder for 0 of 6752, Richland Two Charter High, 
listed https://www.richland2.org/charterhigh/...
Found this folder: 001 Richland Two Charter High

Finding folder for 1 of 6752, POLK STATE COLLEGE COLLEGIATE HIGH SCHOOL, 
listed https://www.polk.edu/lakeland-gateway-to-college-high-school/...
Found this folder: 002 Polk State College Collegiate High School

Finding folder for 2 of 6752, River City Scholars Charter Academy, 
listed https://www.nhaschools.com/schools/rivercity/Pages/default.aspx...
Found this folder: 003 River City Scholars Charter Academy

Finding folder for 3 of 6752, Detroit Enterprise Academy, 
listed https://www.nhaschools.com/schools/detroitenterprise...
Found this folder: 004 Detroit Enterprise Academy

Finding folder for 4 of 6752, MINISINAAKWAANG LEADERSHIP ACADEMY, 
listed https://www.minisacademy.org...
Found this folder: 110 Minisinaakwaang Leadership Academy

Finding folder for 5 of 6752, Lighthouse Community Sch Inc, 
listed https://www.lys.org/ser

Found this folder: 264 Kipp Aspire Academy

Finding folder for 247 of 6752, KIPP GENERATIONS COLLEGIATE, 
listed http://kipphouston.org/generations/...
Found this folder: 265 Kipp Generations Collegiate

Finding folder for 248 of 6752, Kanu o ka Aina - New Century PCS, 
listed http://kalo.org/...
Found this folder: 266 Kanu O Ka Aina - New Century Pcs

Finding folder for 249 of 6752, JAMAA LEARNING CENTER, 
listed http://jamaalearningcenter.org/...
Found this folder: 267 Jamaa Learning Center

Finding folder for 250 of 6752, iSUCCEED VIRTUAL HIGH SCHOOL, 
listed http://isucceed.insightschools.net/...
Found this folder: 268 Isucceed Virtual High School

Finding folder for 251 of 6752, Innovations Academy, 
listed http://innovationsacademy.org/...
Found this folder: 269 Innovations Academy

Finding folder for 252 of 6752, HARMONY SCHOOL OF SCIENCE - HOUSTON, 
listed http://hsshouston.org/...
Found this folder: 271 Harmony School Of Science - Houston

Finding folder for 253 of 6752, Highl


Finding folder for 695 of 6752, Arizona Preparatory Academy, 
listed ...
Found this folder: 

Finding folder for 696 of 6752, Arizona School For The Arts, 
listed ...
Found this folder: 

Finding folder for 697 of 6752, Arizona Virtual Academy, 
listed ...
Found this folder: 

Finding folder for 698 of 6752, ARKANSAS ARTS ACADEMY ELEMENTARY/MIDDLE SCHOOL, 
listed ...
Found this folder: 

Finding folder for 699 of 6752, ARKANSAS ARTS ACADEMY HIGH SCHOOL, 
listed ...
Found this folder: 

Finding folder for 700 of 6752, ARKANSAS VIRTUAL ACADEMY, 
listed ...
Found this folder: 

Finding folder for 701 of 6752, ARKANSAS VIRTUAL ACADEMY HIGH SCHOOL, 
listed ...
Found this folder: 

Finding folder for 702 of 6752, ARKANSAS VIRTUAL ACADEMY JR, 
listed ...
Found this folder: 

Finding folder for 703 of 6752, ARLINGTON CLASSICS ACADEMY - MIDDLE, 
listed ...
Found this folder: 

Finding folder for 704 of 6752, ARLINGTON CLASSICS ACADEMY - PRI, 
listed ...
Found this folder: 

Finding folder for 

Found this folder: 

Finding folder for 1201 of 6752, Capistrano Connections Academy, 
listed ...
Found this folder: 

Finding folder for 1202 of 6752, Capital Area School for the Arts Charter, 
listed ...
Found this folder: 

Finding folder for 1203 of 6752, Capital City HS PCS, 
listed ...
Found this folder: 

Finding folder for 1204 of 6752, Capital City Middle School PCS, 
listed ...
Found this folder: 

Finding folder for 1205 of 6752, Capital High School, 
listed ...
Found this folder: 

Finding folder for 1206 of 6752, Capitol Collegiate Academy, 
listed ...
Found this folder: 

Finding folder for 1207 of 6752, Capitol High School, 
listed ...
Found this folder: 

Finding folder for 1208 of 6752, Capitol West Academy, 
listed ...
Found this folder: 

Finding folder for 1209 of 6752, Capri Elementary, 
listed ...
Found this folder: 

Finding folder for 1210 of 6752, CAPROCK ACADEMY, 
listed ...
Found this folder: 

Finding folder for 1211 of 6752, CAPSTONE ACADEMY, 
listed ...
Fo

Found this folder: 

Finding folder for 1514 of 6752, CLOVERDALE AEROSPACE TECH CHAR, 
listed ...
Found this folder: 

Finding folder for 1515 of 6752, Clovis Online Charter, 
listed ...
Found this folder: 

Finding folder for 1516 of 6752, Clubview Elementary School, 
listed ...
Found this folder: 

Finding folder for 1517 of 6752, Coastal Empire Montessori Charter School, 
listed ...
Found this folder: 

Finding folder for 1518 of 6752, Coastal Grove Charter, 
listed ...
Found this folder: 

Finding folder for 1519 of 6752, Coastal Leadership Academy, 
listed ...
Found this folder: 

Finding folder for 1520 of 6752, Coastal Montessori Charter, 
listed ...
Found this folder: 

Finding folder for 1521 of 6752, COASTAL VILLAGE EL, 
listed ...
Found this folder: 

Finding folder for 1522 of 6752, COASTAL VILLAGE MIDDLE, 
listed ...
Found this folder: 

Finding folder for 1523 of 6752, Coburg Community Charter School, 
listed ...
Found this folder: 

Finding folder for 1524 of 6752, Coche

...

Found this folder: 

Finding folder for 4648 of 6752, Opportunities for Learning - Santa Clarita, 
listed ...
Found this folder: 

Finding folder for 4649 of 6752, OPPORTUNITY CHARTER SCHOOL, 
listed ...
Found this folder: 

Finding folder for 4650 of 6752, Optimist Charter, 
listed ...
Found this folder: 

Finding folder for 4651 of 6752, Optimum Learning Environment Charter School, 
listed ...
Found this folder: 

Finding folder for 4652 of 6752, Options Charter School - Carmel, 
listed ...
Found this folder: 

Finding folder for 4653 of 6752, Options Charter School Noblesville, 
listed ...
Found this folder: 

Finding folder for 4654 of 6752, Options for Youth San Gabriel, 
listed ...
Found this folder: 

Finding folder for 4655 of 6752, Options for Youth-Burbank Charter, 
listed ...
Found this folder: 

Finding folder for 4656 of 6752, Options for Youth-San Bernardino, 
listed ...
Found this folder: 

Finding folder for 4657 of 6752, Options for Youth-San Juan, 
listed ...
Found th

Found this folder: 

Finding folder for 4998 of 6752, PREMIER H S OF LAREDO, 
listed ...
Found this folder: 

Finding folder for 4999 of 6752, PREMIER H S OF LEWISVILLE, 
listed ...
Found this folder: 

Finding folder for 5000 of 6752, PREMIER H S OF LUBBOCK, 
listed ...
Found this folder: 

Finding folder for 5001 of 6752, PREMIER H S OF MIDLAND, 
listed ...
Found this folder: 

Finding folder for 5002 of 6752, PREMIER H S OF MISSION, 
listed ...
Found this folder: 

Finding folder for 5003 of 6752, PREMIER H S OF NORTH AUSTIN, 
listed ...
Found this folder: 

Finding folder for 5004 of 6752, PREMIER H S OF PALMVIEW, 
listed ...
Found this folder: 

Finding folder for 5005 of 6752, PREMIER H S OF RICHARDSON, 
listed ...
Found this folder: 

Finding folder for 5006 of 6752, PREMIER H S OF SAN ANTONIO, 
listed ...
Found this folder: 

Finding folder for 5007 of 6752, PREMIER H S OF SAN JUAN, 
listed ...
Found this folder: 

Finding folder for 5008 of 6752, PREMIER H S OF SOUTH IRVING, 


Found this folder: 

Finding folder for 5507 of 6752, SETTLEMENT HOME, 
listed ...
Found this folder: 

Finding folder for 5508 of 6752, Seven Generations Charter School, 
listed ...
Found this folder: 

Finding folder for 5509 of 6752, Seven Hills Charter School, 
listed ...
Found this folder: 

Finding folder for 5510 of 6752, SEVEN HILLS PREP ACADEMY UPPER, 
listed ...
Found this folder: 

Finding folder for 5511 of 6752, SEVEN HILLS PREPARATORY ACADEMY, 
listed ...
Found this folder: 

Finding folder for 5512 of 6752, Shabazz International Chrtr Schls, 
listed ...
Found this folder: 

Finding folder for 5513 of 6752, Shared Journeys, 
listed ...
Found this folder: 

Finding folder for 5514 of 6752, Shasta Charter Academy, 
listed ...
Found this folder: 

Finding folder for 5515 of 6752, Sheboygan Leadership Academy, 
listed ...
Found this folder: 

Finding folder for 5516 of 6752, SHEELER HIGH CHARTER, 
listed ...
Found this folder: 

Finding folder for 5517 of 6752, Sheila Skip No

Found this folder: 

Finding folder for 6055 of 6752, The New Standard Academy, 
listed ...
Found this folder: 

Finding folder for 6056 of 6752, The Next Frontier Academy, 
listed ...
Found this folder: 

Finding folder for 6057 of 6752, The Next Step PCS, 
listed ...
Found this folder: 

Finding folder for 6058 of 6752, The North Carolina Leadership Academy, 
listed ...
Found this folder: 

Finding folder for 6059 of 6752, The O'Farrell Charter, 
listed ...
Found this folder: 

Finding folder for 6060 of 6752, THE OBAMA ACADEMY FOR BOYS, 
listed ...
Found this folder: 

Finding folder for 6061 of 6752, The Odyssey Preparatory Academy, 
listed ...
Found this folder: 

Finding folder for 6062 of 6752, The Odyssey Preparatory Academy  - Casa Grande, 
listed ...
Found this folder: 

Finding folder for 6063 of 6752, The Odyssey Preparatory Academy Goodyear, 
listed ...
Found this folder: 

Finding folder for 6064 of 6752, THE ODYSSEY SCHOOL, 
listed ...
Found this folder: 

Finding folder

Found this folder: 

Finding folder for 6567 of 6752, West Phoenix High School, 
listed ...
Found this folder: 

Finding folder for 6568 of 6752, West Point Elementary School, 
listed ...
Found this folder: 

Finding folder for 6569 of 6752, West Preparatory Academy, 
listed ...
Found this folder: 

Finding folder for 6570 of 6752, West Sacramento Early College Prep Charter, 
listed ...
Found this folder: 

Finding folder for 6571 of 6752, WEST SIDE SUMMIT CHARTER SCHOOL, 
listed ...
Found this folder: 

Finding folder for 6572 of 6752, WEST UNIVERSITY CHARTER HIGH, 
listed ...
Found this folder: 

Finding folder for 6573 of 6752, West Village Academy, 
listed ...
Found this folder: 

Finding folder for 6574 of 6752, Westchester Secondary Charter, 
listed ...
Found this folder: 

Finding folder for 6575 of 6752, WESTERN ACADEMY CHARTER SCHOOL, 
listed ...
Found this folder: 

Finding folder for 6576 of 6752, Western Center Academy, 
listed ...
Found this folder: 

Finding folder for 65

## Parsing soup for ALL TEXT

In [145]:
# This is how we add together the text from all the HTML files:
print('\n\n\n'.join([format_string(str(testtext)), format_string("b'Here's another example of school site text.")]))

Search form Main menu Our Mission and Philosophy English Our Mission: The mission of Acacia Middle Charter School is to provide a nurturing and challenging learning community to the greater Stockton area. Students will experience learning in ways that will allow them to both understand and improve their world.\xc2\xa0Our exceptional teachers inspire students to seek deep, meaningful understanding; to develop strong academic skills; and to become creative and independent thinkers. A rapidly changing global environment requires that Acacia students acquire knowledge and develop 21st century skills, while learning to define and solve problems creatively. Acacia Middle cultivates compassion, fairness, respect, and individuality within a community that celebrates diversity, authenticity, and service. Our Philosophy: Acacia Middle embodies the following key assumptions in philosophy and daily practice: learning is the young person\xe2\x80\x99s pathway to personal growth school should be both

In [146]:
def parse_alltext(data_location, folder_name, URL_var, junk_list, keep_list): 
    
    # Note: Need to sort folders (in data_location) and schools (in list_of_dicts) on SAME LIST of URLs!
    # can index folder location to dictionary because dict["URL"]=folder_name!
    
    print("Parsing folder " + str(folder_name) + "...")
    
    folder_path = data_location + folder_name + "/" # Create folder path using data directory and folder name
    all_text = "" # Initialize big string of website text
    parsed = [] # Initialize list of parsed HTML files
        
    for fname in glob(folder_path + "**", recursive=True): # Loop through each file in the directory.
        
        try:
            if not fname.endswith("index.html"): # Only parsing index.html files for now
                continue
            if bool(BeautifulSoup(open(fname), "html.parser").find())==True: # Check if file has HTML; if so, then parse it!
                #print("Parsing HTML in " + fname + "...")
                soup = BeautifulSoup(open(fname), "lxml")
                
                ftext = keep_text(fname, junk_list) # extract all text from file, excluding style, etc. elements defined in junk_list
                all_text = "\n\n\n".join([str(all_text), format_string(str(ftext))]) # add ripped text from file to big string, all_text
                
                parsed.append(str(fname)) # add file name to list of files that have been parsed
                return (all_text, parsed) # Only parse one index.html file per school
            else:
                pass
                
        except:
            pass
        
    print("Success! Returning all website text from " + str(URL_var) + "...\n")
    return(all_text, parsed)
        
        
    # look in each folder for .html, read in the text, sub ' for b' if needed at start string
    # join together all the text strings, add that as value to school dict, analyze as before.
    # keep out all sites with only ONE HTML file

In [147]:
# Parse text from website for each school in sample using the parse_text function

k=0 # initialize iterator

for school in sample: # loop through sample
    print("Parsing text for " + str(k) + " of " + str(len(sample)) + ", " + school["SCH_NAME"] + ", \nlisted " + str(school["MANUAL_URL"]) + "...")
    
    school["parsed_files"] = []
    if school["folder_name"] != '':

        try:
            school["alltext"], school["parsed_files"] = \
            parse_alltext(data_loc, school["folder_name"], school["MANUAL_URL"], junkName, correctName)
        except:
            print("ERROR!")
            school["all_text"] = ''

    else:
        print("No folder of scraped web data to parse!")
        school["alltext"] = ''
    
    k+=1


# print(sample[0]["alltext"])

Parsing text for 0 of 6752, Richland Two Charter High, 
listed https://www.richland2.org/charterhigh/...
Parsing folder 001 Richland Two Charter High...
Parsing text for 1 of 6752, POLK STATE COLLEGE COLLEGIATE HIGH SCHOOL, 
listed https://www.polk.edu/lakeland-gateway-to-college-high-school/...
Parsing folder 002 Polk State College Collegiate High School...
Parsing text for 2 of 6752, River City Scholars Charter Academy, 
listed https://www.nhaschools.com/schools/rivercity/Pages/default.aspx...
Parsing folder 003 River City Scholars Charter Academy...
Success! Returning all website text from https://www.nhaschools.com/schools/rivercity/Pages/default.aspx...

Parsing text for 3 of 6752, Detroit Enterprise Academy, 
listed https://www.nhaschools.com/schools/detroitenterprise...
Parsing folder 004 Detroit Enterprise Academy...
Success! Returning all website text from https://www.nhaschools.com/schools/detroitenterprise...

Parsing text for 4 of 6752, MINISINAAKWAANG LEADERSHIP ACADEMY, 


Parsing text for 52 of 6752, ST HOPE LEADERSHIP ACADEMY CHARTER SCHOOL, 
listed http://www.sthopeleadershipacademy.org/...
Parsing folder 052 St Hope Leadership Academy Charter School...
Parsing text for 53 of 6752, ST CROIX PREPARATORY ACADEMY MIDDLE, 
listed http://www.stcroixprep.org/...
Parsing folder 053 St Croix Preparatory Academy Middle...
Parsing text for 54 of 6752, Stony Point Academy, 
listed http://www.spacharter.org/...
Parsing folder 054 Stony Point Academy...
Parsing text for 55 of 6752, Southside Academy, 
listed http://www.southsideedu.org/...
Parsing folder 055 Southside Academy...
Parsing text for 56 of 6752, South Pointe High School, 
listed http://www.southpointehs.com/...
Parsing folder 056 South Pointe High School...
Parsing text for 57 of 6752, SOUTH BRONX CLASSICAL CHARTER SCHOOL II, 
listed http://www.southbronxclassical.org/...
Parsing folder 057 South Bronx Classical Charter School Ii...
Parsing text for 58 of 6752, Somerset Preparatory Academy PCS, 
listed

Parsing text for 108 of 6752, MATER BRICKELL PREPARATORY ACADEMY, 
listed http://www.materacademy.com/project/mater-brickell-preparatory-academy-k-8/...
Parsing folder 114 Mater Brickell Preparatory Academy...
Parsing text for 109 of 6752, MATCH Charter Public School, 
listed http://www.matchschool.org/...
Parsing folder 115 Match Charter Public School...
Parsing text for 110 of 6752, Marshall Academy, 
listed http://www.marshallacademy.org/...
Parsing folder 116 Marshall Academy...
Parsing text for 111 of 6752, RICARDO FLORES MAGON ACADEMY, 
listed http://www.magonacademy.org/...
Parsing folder 117 Ricardo Flores Magon Academy...
Parsing text for 112 of 6752, Magellan Charter, 
listed http://www.magellancharter.org/...
Parsing folder 118 Magellan Charter...
Parsing text for 113 of 6752, Lockhurst Drive Charter Elementary, 
listed http://www.lockhurstelementary.com/...
Parsing folder 119 Lockhurst Drive Charter Elementary...
Parsing text for 114 of 6752, Live Oak Charter, 
listed http:

Parsing text for 160 of 6752, E.C.H.O. CHARTER SCHOOL, 
listed http://www.echo.charter.k12.mn.us/...
Parsing folder 180 E.C.H.O. Charter School...
Parsing text for 161 of 6752, East Preparatory Academy, 
listed http://www.eastprep.org/...
Parsing folder 181 East Preparatory Academy...
Parsing text for 162 of 6752, Eagle Academy, 
listed http://www.eagletoledo.com/...
Parsing folder 182 Eagle Academy...
Parsing text for 163 of 6752, EAGLE RIDGE ACADEMY CHARTER SCHOOL, 
listed http://www.eagleridgeacademy.net/...
Parsing folder 183 Eagle Ridge Academy Charter School...
Parsing text for 164 of 6752, E-Institute at Surprise, 
listed http://www.e-institute.us/locations/surprise/...
Parsing folder 184 E-Institute At Surprise...
Parsing text for 165 of 6752, E-Institute at Grovers, 
listed http://www.e-institute.us/...
Parsing folder 185 E-Institute At Grovers...
Parsing text for 166 of 6752, Detroit Leadership Academy Middle/High, 
listed http://www.dlachampion.org...
No folder of scraped we

Parsing text for 209 of 6752, Academic and Career Education Academy, 
listed http://www.aceaofmidland.org/...
Parsing folder 229 Academic And Career Education Academy...
Parsing text for 210 of 6752, Academy of Tucson Middle School, 
listed http://www.academyoftucson.com/middle-school/middle-school-home...
Parsing folder 230 Academy Of Tucson Middle School...
Success! Returning all website text from http://www.academyoftucson.com/middle-school/middle-school-home...

Parsing text for 211 of 6752, Academy Charter High School, 
listed http://www.academycharterhs.org/pages/mainpg...
Parsing folder 231 Academy Charter High School...
Success! Returning all website text from http://www.academycharterhs.org/pages/mainpg...

Parsing text for 212 of 6752, Academy of Arts and Sciences: Oxnard & Ventura, 
listed http://www.aascalifornia.org/...
Parsing folder 233 Academy Of Arts And Sciences: Oxnard & Ventura...
Parsing text for 213 of 6752, Millennium Community School, 
listed http://www.2kschool

...

No folder of scraped web data to parse!
Parsing text for 2473 of 6752, George D Warriner Middle, 
listed ...
No folder of scraped web data to parse!
Parsing text for 2474 of 6752, George Ellery Hale Charter Academy, 
listed ...
No folder of scraped web data to parse!
Parsing text for 2475 of 6752, GEORGE GERVIN ACADEMY, 
listed ...
No folder of scraped web data to parse!
Parsing text for 2476 of 6752, George Gervin Prep Academy, 
listed ...
No folder of scraped web data to parse!
Parsing text for 2477 of 6752, GEORGE I SANCHEZ H S, 
listed ...
No folder of scraped web data to parse!
Parsing text for 2478 of 6752, GEORGE M KOZMETSKY SCHOOL, 
listed ...
No folder of scraped web data to parse!
Parsing text for 2479 of 6752, George V. Voinovich Reclamation Academy, 
listed ...
No folder of scraped web data to parse!
Parsing text for 2480 of 6752, GEORGE WASHINGTON ACADEMY, 
listed ...
No folder of scraped web data to parse!
Parsing text for 2481 of 6752, George Washington Carver Elementary

No folder of scraped web data to parse!
Parsing text for 3669 of 6752, Leadership Academy of Nevada, 
listed ...
No folder of scraped web data to parse!
Parsing text for 3670 of 6752, LEADERSHIP ACADEMY WEST, 
listed ...
No folder of scraped web data to parse!
Parsing text for 3671 of 6752, Leadership and Entrepreneurship Public Charter High School, 
listed ...
No folder of scraped web data to parse!
Parsing text for 3672 of 6752, Leadership High, 
listed ...
No folder of scraped web data to parse!
Parsing text for 3673 of 6752, LEADERSHIP LEARNING ACADEMY, 
listed ...
No folder of scraped web data to parse!
Parsing text for 3674 of 6752, LEADERSHIP PREP SCHOOL, 
listed ...
No folder of scraped web data to parse!
Parsing text for 3675 of 6752, LEADERSHIP PREPARATORY BEDFORD STUYVESANT CHARTER SCHOOL, 
listed ...
No folder of scraped web data to parse!
Parsing text for 3676 of 6752, LEADERSHIP PREPARATORY BROWNSVILLE CHARTER SCHOOL, 
listed ...
No folder of scraped web data to parse!
Pa

No folder of scraped web data to parse!
Parsing text for 4380 of 6752, NEW MILLENNIUM CHARTER SCHOOL, 
listed ...
No folder of scraped web data to parse!
Parsing text for 4381 of 6752, New Millennium Secondary, 
listed ...
No folder of scraped web data to parse!
Parsing text for 4382 of 6752, NEW NEIGHBOR CAMPUS, 
listed ...
No folder of scraped web data to parse!
Parsing text for 4383 of 6752, New Opportunities Charter, 
listed ...
No folder of scraped web data to parse!
Parsing text for 4384 of 6752, New Orleans Charter Science and Mathematics HS, 
listed ...
No folder of scraped web data to parse!
Parsing text for 4385 of 6752, New Orleans Military/Maritime Academy, 
listed ...
No folder of scraped web data to parse!
Parsing text for 4386 of 6752, New Paradigm College Prep, 
listed ...
No folder of scraped web data to parse!
Parsing text for 4387 of 6752, New Paradigm Glazer Academy, 
listed ...
No folder of scraped web data to parse!
Parsing text for 4388 of 6752, New Paradigm Lovi

Parsing text for 4934 of 6752, Pleasant Community Digital, 
listed ...
No folder of scraped web data to parse!
Parsing text for 4935 of 6752, Pleasant Education Academy, 
listed ...
No folder of scraped web data to parse!
Parsing text for 4936 of 6752, Plumas Charter, 
listed ...
No folder of scraped web data to parse!
Parsing text for 4937 of 6752, Plymouth Educational Center, 
listed ...
No folder of scraped web data to parse!
Parsing text for 4938 of 6752, Plymouth Educational Center Preparatory High School, 
listed ...
No folder of scraped web data to parse!
Parsing text for 4939 of 6752, Plymouth Scholars Charter Academy, 
listed ...
No folder of scraped web data to parse!
Parsing text for 4940 of 6752, POCATELLO COMMUNITY CHARTER, 
listed ...
No folder of scraped web data to parse!
Parsing text for 4941 of 6752, Pocono Mountain Charter School, 
listed ...
No folder of scraped web data to parse!
Parsing text for 4942 of 6752, PODER Academy, 
listed ...
No folder of scraped web dat

Parsing text for 5800 of 6752, STONEBRIDGE WORLD SCHOOL, 
listed ...
No folder of scraped web data to parse!
Parsing text for 5801 of 6752, STREAM Charter, 
listed ...
No folder of scraped web data to parse!
Parsing text for 5802 of 6752, STRIDE ACADEMY CHARTER SCHOOL, 
listed ...
No folder of scraped web data to parse!
Parsing text for 5803 of 6752, STRIDE ACADEMY MIDDLE SCHOOL, 
listed ...
No folder of scraped web data to parse!
Parsing text for 5804 of 6752, STRIVE PREP - EXCEL, 
listed ...
No folder of scraped web data to parse!
Parsing text for 5805 of 6752, STRIVE PREP - FEDERAL, 
listed ...
No folder of scraped web data to parse!
Parsing text for 5806 of 6752, STRIVE PREP - GVR, 
listed ...
No folder of scraped web data to parse!
Parsing text for 5807 of 6752, STRIVE PREP - LAKE, 
listed ...
No folder of scraped web data to parse!
Parsing text for 5808 of 6752, STRIVE PREP - MONTBELLO, 
listed ...
No folder of scraped web data to parse!
Parsing text for 5809 of 6752, STRIVE PREP

In [148]:
sample[7]["alltext"]

'\n\n\n17160 Van Gogh St. Granada Hills, CA 91344 Phone: 818.360.2141 Fax: 818.831.9081 Office hours: 7:30am-4:30pm Principal: Pamela Merloni SAA: Raquel Acosta Office Tech: Nancy Zweben Games for Students Teacher Resources Cafeteria Menu Friends of Van Gogh PTO Principal\'s Corner Calendar 2016 8/16 : School Starts 9/2 : Admissions Day 9/5 : Labor Day 10/3 : Unassigned Holiday 10/12 : Unassigned Holiday 11/11 : Veteran\'s Day 11/21-23 : Unassigned Days 11/24-25 : Thanksgiving Break 2017 12/19/16-1/6/17 : Winter Break 1/16 : M.L.King Jr. Day 2/20 : President\'s Day 3/31 : Cesar E. Chavez Day 4/10-4/14 : Spring Break 5/29 : Memorial Day 6/9 : Last Day of School Click on the calendar to view other important school events and meetings. Click here to view Principal Merloni\'s newsletter message for the opening school year. Good Evening Van Gogh Staff and Families, I hope that you\'ve had a wonderful, long weekend! This week we begin our Valet Program with volunteers from Ms. Haratian\'s cl

In [149]:
list(sample[200]["folder_name"])

[]

In [150]:
# Number of files that were parsed for first school
print(len(sample[200]["parsed_files"]))

0


In [151]:
import csv
def sample_to_csv(sample):
    with open('output.csv', 'w', newline='') as csvfile:
        
        fieldnames = list(sample[0].keys())
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for s in sample:
            writer.writerow(s)
            

In [152]:
sample_to_csv(sample)

## Parsing soup for keywords

In [30]:
# Define lists of keywords for each category of interest, used to group the text
mission = ['mission',' vision ', 'vision:', 'mission:', 'our purpose', 'our ideals', 'ideals:', 'our cause', 'cause:', 'goals', 'objective']
curriculum = ['curriculum', 'curricular', 'program', 'method', 'pedagogy', 'pedagogical', 'approach', 'model', 'system', 'structure']
philosophy = ['philosophy', 'philosophical', 'beliefs', 'believe', 'principles', 'creed', 'credo', 'value',  'moral', 'Values']
history = ['history', 'our story', 'the story', 'school story', 'background', 'founding', 'founded', 'established', 'establishment', 'our school began', 'we began', 'doors opened', 'school opened']
general =  ['about us', 'our school', 'who we are', 'overview', 'general information', 'our identity', 'profile', 'highlights']

keyValues = [mission, curriculum, philosophy, history, general]
nodeParams = ['mission', 'curriculum', 'philosophy', 'history', 'general']

In [31]:
# This helper function creates a list of tags (with text) for a given keyWord:
def findTags(soup, keyWord):
    tagList = []
        
    for elem in soup(text=re.compile(keyWord)):
        tagList.append(elem.parent)
            
    return tagList

In [32]:
mission_tags = findTags(soup, "mission")
mission_tags

[<script type="text/javascript">
 <!--//--><![CDATA[//><!--
 jQuery.extend(Drupal.settings, {"basePath":"\/","pathPrefix":"","ajaxPageState":{"theme":"lvcp","theme_token":"l0EddeJX-Q2S2jRbcsDnU_wolEGc000-GhODeFykpQI","js":{"misc\/jquery.js":1,"misc\/jquery.once.js":1,"misc\/drupal.js":1,"sites\/all\/libraries\/colorbox\/jquery.colorbox-min.js":1,"sites\/all\/modules\/colorbox\/js\/colorbox.js":1,"sites\/all\/modules\/colorbox\/styles\/default\/colorbox_default_style.js":1,"sites\/all\/modules\/field_group\/field_group.js":1,"sites\/all\/themes\/noodle\/base\/noodle.js":1,"sites\/all\/libraries\/superfish\/superfish.js":1,"sites\/all\/libraries\/superfish\/jquery.hoverIntent.minified.js":1,"sites\/all\/libraries\/superfish\/supersubs.js":1,"sites\/all\/modules\/superfish\/superfish.js":1},"css":{"modules\/system\/system.base.css":1,"modules\/system\/system.menus.css":1,"modules\/system\/system.messages.css":1,"modules\/system\/system.theme.css":1,"sites\/all\/modules\/date\/date_api\/da

In [35]:
def parse_keytext(data_location, folder_name, URL_var, junk_list): 
    
    # Note: Need to sort folders (in data_location) and schools (in list_of_dicts) on SAME LIST of URLs!
    # can index folder location to dictionary because dict["URL"]=folder_name!
    
    print("Parsing folder " + str(folder_name) + "...")
    
    folder_path = data_location + folder_name + "/" # Create folder path using data directory and folder name
    all_text = "" # Initialize big string of website text
    parsed = [] # Initialize list of parsed HTML files
        
    for fname in glob(folder_path + "**", recursive=True): # Loop through each file in the directory
        
        try:
            
            if bool(BeautifulSoup(open(fname), "html.parser").find())==True: # Check if file has HTML; if so, then parse it!
                #print("Parsing HTML in " + fname + "...")
                soup = BeautifulSoup(open(fname), "lxml")
                
                ftext = rip_text(fname, junk_list) # extract all text from file, excluding style, etc. elements defined in junk_list
                
                all_text = "\n\n\n".join([str(all_text), format_string(str(ftext))]) # add ripped text from file to big string, all_text
                
                parsed.append(str(fname)) # add file name to list of files that have been parsed
            
            else:
                pass
                
        except:
            pass
        
    print("Success! Returning all website text from " + str(URL_var) + "...\n")
    return(all_text, parsed)
        
        
    # look in each folder for .html, read in the text, sub ' for b' if needed at start string
    # join together all the text strings, add that as value to school dict, analyze as before.
    # keep out all sites with only ONE HTML file

In [36]:
# Parse text from website for each school in sample using the parse_text function

k=0 # initialize iterator

for school in sample[:10]: # loop through sample
    print("Parsing text for " + str(k) + " of " + str(len(sample)) + ", " + school["SCH_NAME"] + ", \nlisted " + str(school["MANUAL_URL"]) + "...")
    
    if school["folder_name"] != '':
        
        try:
            school["parsed_files"] = []
            school["alltext"], school["parsed_files"] = \
            parse_alltext(data_loc, school["folder_name"], school["MANUAL_URL"], junkName)
        except:
            print("ERROR!")
            school["all_text"] = ''
    
    else:
        print("No folder of scraped web data to parse!")
        school["alltext"] = ''
    
    k+=1


# print(sample[0]["alltext"])

Parsing text for 0 of 6752, Richland Two Charter High, 
listed https://www.richland2.org/charterhigh/...
ERROR!
Parsing text for 1 of 6752, POLK STATE COLLEGE COLLEGIATE HIGH SCHOOL, 
listed https://www.polk.edu/lakeland-gateway-to-college-high-school/...
ERROR!
Parsing text for 2 of 6752, River City Scholars Charter Academy, 
listed https://www.nhaschools.com/schools/rivercity/Pages/default.aspx...
ERROR!
Parsing text for 3 of 6752, Detroit Enterprise Academy, 
listed https://www.nhaschools.com/schools/detroitenterprise...
ERROR!
Parsing text for 4 of 6752, MINISINAAKWAANG LEADERSHIP ACADEMY, 
listed https://www.minisacademy.org...
ERROR!
Parsing text for 5 of 6752, Lighthouse Community Sch Inc, 
listed https://www.lys.org/services/lighthouse-community-school/...
ERROR!
Parsing text for 6 of 6752, Westlake Charter Middle, 
listed https://westlakecharter.com...
ERROR!
Parsing text for 7 of 6752, Van Gogh Charter, 
listed https://vangoghcs-lausd-ca.schoolloop.com/...
ERROR!
Parsing text

## BETA/ FOR REFERENCE (mostly from this notebook)

In [80]:
## Create HTML soup
# AKCAN'S VERSION:
html_page = urllib.request.urlopen(url)
soup = BeautifulSoup(html_page, "lxml")

# THAO'S VERSION:
html_page = requests.get(url, auth=HTTPDigestAuth('user', 'pass'), headers= {'User-Agent':"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36"})
soup = BeautifulSoup(html_page.text, "lxml")

NameError: name 'url' is not defined

In [49]:
#(self, url, nodeParams, keyValues)


#creating a list of all the keyword lists
#keyValues = [mission, curriculum, philosophy, history, target, resources, orgfactor]
keyValues = [mission, curriculum, philosophy, history, general]

#creating a list of key names for the JSON node that correspond to the keyValues list
#nodeParams = ['mission', 'curriculum', 'philosophy', 'history', 'target', 'resources' , 'organizational_factors']
nodeParams = ['mission', 'curriculum', 'philosophy', 'history', 'general']
testedLinks = {}

#print(links[0])



#constructor = school.WebsiteData('https://www.richland2.org/charterhigh/', nodeParams, keyValues)
#constructor.createJsonNode()
#print(constructor.URL)

def tester(links, keyValues, nodeParams):
    testedLinks[links[5]] = school.WebsiteData(links[5], nodeParams, keyValues).createJsonNode()
    #for i in range(5, 6):
        #testedLinks[links[i]] = school.WebsiteData(links[i], nodeParams, keyValues).createJsonNode()
tester(links, keyValues, nodeParams)

NameError: name 'links' is not defined

## TESTING (from EncapsulatedSchoolObject.ipynb)

In [82]:
#Finds all tags within a BeautifulSoup object that's text contains a certain keyword and returns a list of these tags.
def findTags(soup, keyWord):
    tagList = []
        
    for elem in soup(text=re.compile(keyWord)):
        tagList.append(elem.parent)
            
    return tagList

In [83]:
#Utilizes a BeautifulSoup object (soup) to scrape all text that contains the given keyword. Filters for certain tags and does
#not take text from solely header (h) tags, instead takes text from next filtered tag after the header.  
def keyWordScraper(soup, keyWord):
    searchTxt = ''
    correctName = ['p', 'li', 'table', 'ul', 'ol']
    tagList = findTags(soup, keyWord)
    #print(tagList)
    
    #This loop goes through each tag and rips the text from that tag. If that tag is a header, then the text from the next 
    #useful tag is ripped instead. All text is saved in searchTxt
    for tag in tagList:
        #print(tag, "\n")
        if 'h' in tag.name:
            typeTag = type(soup.find('li'))
            current = tag.next_sibling
            while current != None and  not isinstance(current, typeTag) and current.name not in correctName:
                current = current.next_sibling
            if current != None:
                if 'ul' in current.name or 'ol' in current.name:
                    for li in current.findAll('li'):
                        searchTxt = searchTxt + li.text
                else:
                    searchTxt = searchTxt + current.text
        #elif url extension includes the keyWord, grab all the text?
        else:
            if(tag.name in correctName):
                    searchTxt = searchTxt + tag.text
                
        return searchTxt

In [84]:
#Applies the keyWordScraper function to a BeautifulSoup object(soup) for a list of keywords. Returns the text found.
def mapKeywordScraper(soup, keyWords):
    foundTxt = ''
    
    for keyWord in keyWords:
        foundTxt = foundTxt + str(keyWordScraper(soup, keyWord))
    #print(foundTxt)
    return foundTxt

In [91]:
def createJsonNode():
    JSONNode = {}
    
    for x in range(0, len(keyValues)):
        print(keyValues[x], "\n")
        JSONNode[nodeParams[x]] = [mapKeywordScraper(soup, keyValues[x])]
        print(JSONNode)
    return JSONNode

In [92]:
createJsonNode()

['mission', ' vision ', 'vision:', 'mission:', 'our purpose', 'our ideals', 'ideals:', 'our cause', 'cause:', 'goals', 'objective'] 

{'mission': ['NoneNoneNoneNoneNoneNoneNoneNoneNoneNone']}
['curriculum', 'curricular', 'program', 'method', 'pedagogy', 'pedagogical', 'approach', 'model', 'system', 'structure'] 

{'mission': ['NoneNoneNoneNoneNoneNoneNoneNoneNoneNone'], 'curriculum': ['NoneNoneTVLC and the charter schools it operates, Livermore Valley Charter Preparatory High School, Livermore Valley Charter School, Acacia Elementary Charter School. and Acacia Middle Charter School have no affiliations, partnerships or sister-school relationships with any educational programs located in China. NoneNoneNoneNoneNoneNone']}
['philosophy', 'philosophical', 'beliefs', 'believe', 'principles', 'creed', 'credo', 'value', 'moral', 'Values'] 

{'mission': ['NoneNoneNoneNoneNoneNoneNoneNoneNoneNone'], 'curriculum': ['NoneNoneTVLC and the charter schools it operates, Livermore Valley Charter Prep

{'curriculum': ['NoneNoneTVLC and the charter schools it operates, Livermore Valley Charter Preparatory High School, Livermore Valley Charter School, Acacia Elementary Charter School. and Acacia Middle Charter School have no affiliations, partnerships or sister-school relationships with any educational programs located in China. NoneNoneNoneNoneNoneNone'],
 'general': ['NoneNoneNoneNoneNoneNoneNoneNone'],
 'history': ['NoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNone'],
 'mission': ['NoneNoneNoneNoneNoneNoneNoneNoneNoneNone'],
 'philosophy': ['NoneNoneNoneNoneNoneNoneNoneNoneNone']}

## CHUNKS OF CODE

In [81]:
# Input test URL here
url = links[0]
url

NameError: name 'links' is not defined

In [56]:
html_page = urllib.request.urlopen(url)
soup = BeautifulSoup(html_page, "lxml")

NameError: name 'url' is not defined

In [57]:
def findTags(soup, keyWord):
    tagList = []
        
    for elem in soup(text=re.compile(keyWord)):
        tagList.append(elem.parent)
            
    return tagList

In [58]:
tagList = findTags(soup, keyWord)

NameError: name 'keyWord' is not defined

In [59]:
searchTxt = ''
correctName = ['p', 'li', 'table', 'ul', 'ol']
tagList = findTags(soup, keyWord)
    
#This loop goes through each tag and rips the text from that tag. If that tag is a header, then the text from the next 
 #useful tag is ripped instead. All text is saved in searchTxt
for tag in tagList:
    if 'h' in tag.name:
        typeTag = type(soup.find('li'))
        current = tag.next_sibling
        while current != None and  not isinstance(current, typeTag) and current.name not in correctName:
            current = current.next_sibling
        if current != None:
            if 'ul' in current.name or 'ol' in current.name:
                for li in current.findAll('li'):
                    searchTxt = searchTxt + li.text
            else:
                searchTxt = searchTxt + current.text
    #elif url extension includes the keyWord, grab all the text?
    else:
        if(tag.name in correctName):
            searchTxt = searchTxt + tag.text
                
return searchTxt

NameError: name 'keyWord' is not defined